<a href="https://colab.research.google.com/github/DeveloperMarwan/RareSkills_ZK/blob/main/Groth16_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install py_ecc
!python -m pip install galois

import numpy as np
import galois
from py_ecc.bn128 import G1, G2, pairing, add, multiply, eq, curve_order, neg, Z1, field_modulus, G12, Z2, FQ12, is_on_curve, b12, final_exponentiate
from functools import reduce

GF = galois.GF(curve_order)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 33.6 MB/s eta 0:00:00


In [5]:
def adjust_negative_values(arr, curve_order):
  for i in range(arr.shape[0]):
      for j in range(arr.shape[1]):
          if arr[i, j] < 0:
              arr[i, j] = curve_order + arr[i, j]
  return arr

def interpolate_column(col):
  xs = GF(np.array([1,2,3,4]))
  return galois.lagrange_poly(xs, col)

def create_polynomial_t(degree, curve_order):
  t = galois.Poly([1], field=GF)  # Initialize t as the multiplicative identity in the polynomial ring

  for i in range(1, degree + 1):
      # Each factor is (x - i), with i subtracted from curve_order to stay within the field
      factor = galois.Poly([1, curve_order - i], field=GF)
      t *= factor  # Multiply the current t by the new factor

  return t

def inner_product_polynomials_with_witness(polys, witness):
  mul_ = lambda x, y: x * y
  sum_ = lambda x, y: x + y
  return reduce(sum_, map(mul_, polys, witness))

def inner_product(ec_points, coeffs, generator):
  if len(generator) == 2:
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(ec_points, coeffs)), Z1)
  elif len(generator) == 4:
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(ec_points, coeffs)), Z2)

def pad_coefficients(polys):
  max_length = max(len(poly.coeffs) for poly in polys)
  zero_element = GF(0)  # Create a zero element in the Galois field
  padded_coeffs = [list(poly.coeffs) + [zero_element] * (max_length - len(poly.coeffs)) for poly in polys]
  return padded_coeffs

def trusted_setup_ec(degree, u_polys, v_polys, w_polys, t):
  print("trusted_setup_ec - START")

  alpha = GF(8)
  beta = GF(9)
  tau = GF(100)
  delta = GF(51)
  gama = GF(39)

  powers_of_tau_G1 = [multiply(G1, int(tau ** i)) for i in range(degree + 1)]
  powers_of_tau_G2 = [multiply(G2, int(tau ** i)) for i in range(degree + 1)]

  padded_u = pad_coefficients(u_polys)
  padded_u_rev = [padded[::-1] for padded in padded_u]
  A_powers_of_tau_ec = [inner_product(powers_of_tau_G1, padded, G1) for padded in padded_u_rev]
  #print("A_powers_of_tau_ec: ", A_powers_of_tau_ec)

  Alpha_G1 = multiply(G1, int(alpha))

  padded_v = pad_coefficients(v_polys)
  padded_v_rev = [padded[::-1] for padded in padded_v]
  B_powers_of_tau_G1 = [inner_product(powers_of_tau_G1, padded, G1) for padded in padded_v_rev]
  B_powers_of_tau_G2 = [inner_product(powers_of_tau_G2, padded, G2) for padded in padded_v_rev]

  Beta_G1 = multiply(G1, int(beta))
  Beta_G2 = multiply(G2, int(beta))

  padded_w = pad_coefficients(w_polys)
  padded_w_rev = [padded[::-1] for padded in padded_w]

  C_powers_of_tau_ec = []
  for i in range(len(u_polys)):
    u_contribution = inner_product(powers_of_tau_G1, padded_u_rev[i], G1) if i < len(padded_u) else Z1
    v_contribution = inner_product(powers_of_tau_G1, padded_v_rev[i], G1) if i < len(padded_v) else Z1
    w_contribution = inner_product(powers_of_tau_G1, padded_w_rev[i], G1) if i < len(padded_w) else Z1
    C_powers_of_tau_ec.append(add(add(multiply(u_contribution, int(beta)), multiply(v_contribution, int(alpha))), w_contribution))

  # Splitting C_powers_of_tau into public and private parts
  C_powers_of_tau_public_ec = C_powers_of_tau_ec[:l+1]
  C_powers_of_tau_private_ec = C_powers_of_tau_ec[l+1:]

  # Calculating inverses
  delta_inv = pow(int(delta), -1, curve_order)
  gama_inv = pow(int(gama), -1, curve_order)

  # Multiplying elements
  C_powers_of_tau_public_ec = [multiply(x, gama_inv) for x in C_powers_of_tau_public_ec]
  C_powers_of_tau_private_ec = [multiply(x, delta_inv) for x in C_powers_of_tau_private_ec]

  HT_powers_of_tau_ec = [multiply(G1, int(delta_inv * (tau ** i) * t(tau))) for i in range(degree)]

  Delta_G1 = multiply(G1, int(delta))
  Delta_G2 = multiply(G2, int(delta))
  Gamma_G2 = multiply(G2, int(gama))

  return A_powers_of_tau_ec, Alpha_G1, B_powers_of_tau_G1, B_powers_of_tau_G2, Beta_G1, Beta_G2, C_powers_of_tau_public_ec, C_powers_of_tau_private_ec, HT_powers_of_tau_ec, Delta_G1, Delta_G2, Gamma_G2

# 1, out, x, y, v1, v2, v3
L = adjust_negative_values(np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 5, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, -4],
], dtype=object), curve_order)
#print("L: ", L)
print("L.shape: ", L.shape)

R = adjust_negative_values(np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
], dtype=object), curve_order)
#print("R: ", R)

O = adjust_negative_values(np.array([
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 10, -1, -1, -13],
], dtype=object), curve_order)
#print("O: ", O)

L_galois = GF(L)
R_galois = GF(R)
O_galois = GF(O)

x = GF(29)
y = GF(11)
v1 = x * x
v2 = 5 * v1 * x
v3 = y * y

out = v2 + 13*v3 + v1 + GF(curve_order - 10)*y + GF(curve_order - 4)*v3*v1
witness = GF(np.array([1, out, x, y, v1, v2, v3]))
assert all(np.equal(np.matmul(L_galois, witness) * np.matmul(R_galois, witness), np.matmul(O_galois, witness))), "not equal"

u = np.apply_along_axis(interpolate_column, 0, L_galois)
#print("u: ", u)
v = np.apply_along_axis(interpolate_column, 0, R_galois)
#print("v: ", v)
w = np.apply_along_axis(interpolate_column, 0, O_galois)
#print("w: ", w)

t = create_polynomial_t(L.shape[0], curve_order)
#print("t: ", t)

l = 1
r = GF(29)
s = GF(97)

ui = inner_product_polynomials_with_witness(u, witness)
#print("ui: ", ui)
vi = inner_product_polynomials_with_witness(v, witness)
#print("vi: ", vi)
wi = inner_product_polynomials_with_witness(w, witness)
#print("wi: ", wi)

h = (ui * vi - wi) // t
#print("h: ", h)
#print("t: ", t)

assert ui * vi == wi + h * t, "division has a remainder"

(A_powers_of_tau_ec,
 Alpha_G1,
 B_powers_of_tau_G1,
 B_powers_of_tau_G2,
 Beta_G1,
 Beta_G2,
 C_powers_of_tau_public_ec,
 C_powers_of_tau_private_ec,
 HT_powers_of_tau_ec,
 Delta_G1,
 Delta_G2,
 Gamma_G2) = trusted_setup_ec(L.shape[0] - 1, u, v, w, t)

A_G1 = add(add(Alpha_G1, inner_product(A_powers_of_tau_ec, witness, G1)), multiply(Delta_G1, int(r)))
#print("A_ec: ", A_ec)

B_G1 = add(add(Beta_G1, inner_product(B_powers_of_tau_G1, witness, G1)), multiply(Delta_G1, int(s)))

B_G2 = add(add(Beta_G2, inner_product(B_powers_of_tau_G2, witness, G2)), multiply(Delta_G2, int(s)))
#print("B_ec: ", B_ec)

HT_of_tau = inner_product(HT_powers_of_tau_ec, h.coeffs[::-1], G1)
#print("HT_of_tau: ", HT_of_tau)

C_ec = add(add(add(add(inner_product(C_powers_of_tau_private_ec, witness[l+1:], G1), HT_of_tau), multiply(A_G1, int(s))), multiply(B_G1, int(r))), neg(multiply(Delta_G1, int(r * s))))
#print("C_ec: ", C_ec)

public_input_ec = inner_product(C_powers_of_tau_public_ec, witness[:l+1], G1)

LHS = pairing(B_G2, neg(A_G1))
print ("LHS: ", LHS)

pairing_alpha_beta = pairing(Beta_G2, Alpha_G1)
print ("pairing_alpha_beta: ", pairing_alpha_beta)

pairing_C_G2 = pairing(Delta_G2, C_ec)
print ("pairing_C_G2: ", pairing_C_G2)

pairing_public_input = pairing(Gamma_G2, public_input_ec)
print ("pairing_public_input: ", pairing_public_input)

combined_result = LHS * pairing_alpha_beta * pairing_C_G2 * pairing_public_input
print ("combined_result: ", combined_result)

final_result = final_exponentiate(combined_result)
print ("final_result: ", final_result)

assert final_result == FQ12.one()




L.shape:  (4, 7)
trusted_setup_ec - START
LHS:  (16812722631522406151247224517458142713737212983734898988336979699867879031285, 21885192039781704003521128090656957641613899228713539559197475904659167271115, 7060750208252306988294639647481231398038420181592343516970172848264090074062, 17631253100937334846155496436429115958669382738663156723844598386634755612752, 12310599175775344416722762487905294933793757791646387371784336170525554066167, 7365217658704218879053923095100914524207116088547382911781844064770471979886, 15421093953044838339074704368489700794144022139088293118895290415389340524816, 2786373567312510373053990280810923025617260592692777624550699462618642091086, 16044359056128442294252598862509982624691598142790706643025527622050576051629, 15674535038454772492238257166005605249141723261630337455382356484705555050088, 18396401423611148600333454554843925647588650673481723253894167138496658649913, 7791528051885230340236144132631355715383636437921333826854113538384914791289)
pairing